In [12]:
import pandas as pd
import xml.etree.ElementTree as ET

In [13]:
'''
a shortcut to ET.SubElenment(parent, child).text ( and *.text = text )
'''
def add_node(parent, child, text):
    if type(text)==str:
        ET.SubElement(parent, child).text = text
    else:
        ET.SubElement(parent, child)

In [14]:
'''
pretty print xml from http://effbot.org/zone/element-lib.htm#prettyprint
'''
def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [15]:
# load features and variables from csv
features_all = pd.read_csv('./WEAP_default_features.csv')
variables_all = pd.read_csv('./WEAP_default_variables.csv')

In [27]:
# template name
tpl = ET.Element('template_definition')
tree = ET.ElementTree(tpl)
ET.SubElement(tpl, 'template_name').text = 'WEAP template'

In [28]:
'''
add layout
'''
layout = ET.SubElement(tpl, 'layout')
item = ET.SubElement(layout, 'item')
add_node(item, 'name', 'grouping')
value = ET.SubElement(item, 'value')
add_node(value, 'name', 'WEAP')
add_node(value, 'description', 'A template to mimick WEAP features and symbols')
categories = ET.SubElement(value, 'categories')

cats = features_all.category.unique()
for cat in cats:
    category = ET.SubElement(categories, 'category')
    for attr in ['name', 'description', 'displayname']:
        add_node(category, attr, cat)
    groups = ET.SubElement(category, 'groups')
    
    features = features_all[features_all.category==cat]
    for f in features.itertuples():
        group = ET.SubElement(groups, 'group')
        add_node(group, 'name', f.name)
        add_node(group, 'description', f.description)
        add_node(group, 'displayname', f.displayname)
        add_node(group, 'image', f.image)

In [35]:
'''
add resources
'''
resources = ET.SubElement(tpl, 'resources')

features = variables_all.feature.unique()
for feature in features:

    resource = ET.SubElement(resources, 'resource')

    # add resource layout info
    
    
    # add resource info
    add_node(resource, 'type', 'NODE')
    add_node(resource, 'name', feature)
    layout = ET.SubElement(resource, 'layout')
    item = ET.SubElement(layout, 'item')
    add_node(item, 'name', 'image')
    add_node(item, 'value', 'images\\'+feature+'.png')
    item = ET.SubElement(layout, 'item')
    add_node(item, 'name', 'group')
    add_node(item, 'value', feature)
    
    # add variables
    feature_variables = variables_all[variables_all.feature == feature]
    for v in feature_variables.itertuples():
        attr = ET.SubElement(resource, 'attribute')
        add_node(attr, 'name', v.variable)
        add_node(attr, 'default', v.default_value)
        add_node(attr, 'unit', v.unit)
        add_node(attr, 'scale', v.scale)
        add_node(attr, 'per_unit', v.per_unit)
        if v.has_dest=='TRUE':
            ET.SubElement(attr, 'to')
        if v.has_src=='TRUE':
            ET.SubElement(attr, 'from')

In [36]:
indent(tpl)

In [37]:
tree.write('./WEAP template/template/template.xml', xml_declaration=True, encoding='utf-8')